For this assignment, you will publish the complete code, data, and analysis of your final research project. The goal is to demonstrate that you can incorporate all of the human-centered design considerations you learned in this course and create research artifacts that are understandable, impactful, and reproducible.

The final project report is intended to serve as...

A demonstration of an end-to-end scientific research project that incorporates the principles and practices that you learned in this course. 
A rich and detailed portfolio piece that you can use to demonstrate your capabilities as a scientist and an expert data science practitioner.
An answer to a research question that you came up with yourself because you think is both practically useful and scientifically interesting.
An exercise in designing and conducting quality research within a particular set of constraints. 
A successful report will take the form of a well-written, well-executed research study document (plus supplementary resources, see the full requirements below) contained in a folder within your Github repository—the same folder that holds your final project plan (Assignment 5).

Your previous deliverables for the final project proposal and plan (assignments 4 and 5) are part of this report: you are expected to build your report by adding more text (and, obviously, code!) to your existing project plan Jupyter Notebook.

Structure of the report
The report should be structured well, with headings and even sub-headings. Most reports should have a structure similar to this one

Introduction
Background or Related Work
Research questions or hypotheses
Methodology
Findings
Discussion (including Limitations and Implications)
Conclusion
References